In [1]:
## libraries & dataset
import seaborn as sns
import matplotlib.pyplot as plt
import mplcursors
import csv, pandas as pd
from datetime import datetime, timedelta

In [2]:
# files
roadTrafficFiles=['road_traffic_full.csv']

roadTrafficPath='./Road Traffic/'

filepath = roadTrafficPath+roadTrafficFiles[0]

formats = ['%Y-%m-%dT%H:%M:%S', '%m/%d/%Y %H:%M:%S', '%d/%m/%Y %H:%M:%S', '%d/%m/%Y %H:%M']

datetimeFormat = formats[3]

In [ ]:
df = pd.DataFrame([], columns=['case', 'activity', 'time'])
with open(filepath) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    header = []
    for row in csv_reader:
        if line_count == 0:
            line_count += 1
            header = row
            #print("Headers: ", row)
        else:
            df = df.append(pd.DataFrame([row], columns=header),ignore_index=True)
            line_count += 1
#print(df.info())

In [ ]:
##Sorting DataFrame by Case then Time.
df = df.sort_values(['case','time'], ascending=[True, True])
# print(df)

#minute or Hour?
# minuteFlag = True
# divisor = 60 if minuteFlag else 3600
# timeLabel = 'Time (Minutes)' if minuteFlag else'Time (Hour)'

divisor = 1
timeLabel = 'Time (Seconds)'

## group by case
listA = df.groupby(["case"])
links__ = []
timediff__ = []
for cases in df['case'].unique():
    temp__ = []
    time__ = []
    temp__.append(listA.get_group(cases)["activity"])
    time__.append(listA.get_group(cases)["time"]) 
    for idx, val in temp__[0].iteritems():
        nextIndex = idx +1
        if nextIndex not in temp__[0].index:
            break
        else:
            links__.append({"source": val, "target": temp__[0][nextIndex]})
    for idx, val in time__[0].iteritems():
        nextIndex = idx +1
        if nextIndex not in time__[0].index:
            break
        else:
            timeDiffTemp__ = datetime.strptime(time__[0][nextIndex],datetimeFormat) - datetime.strptime(val,datetimeFormat)
            timediff__.append({"time": timeDiffTemp__.total_seconds()/divisor})

print('####')

In [ ]:
for val in range(0,len(links__)):    
    links__[val].update(timediff__[val])
    links__[val].update({'edge': links__[val]['source']+'>'+links__[val]['target']})
# print()

In [ ]:
link = pd.DataFrame(links__)
# print(links__)
link.head()

In [ ]:
print('###### Before Filtering Statistics : ')
print(link['time'].describe())

In [ ]:
## let us define a delta to check if the process has sub-divisions based on the time duration
# time in minutes/seconds; check minuteFlag section
timeDeltaCheck = 0
link = link[(link['time'] > timeDeltaCheck)]

In [ ]:
print('######')
print(link['time'].describe())

In [ ]:
# set a grey background (use sns.set_theme() if seaborn version 0.11.0 or above) 
sns.set(style="darkgrid")
# plt.figure(figsize=(8,3))
# figA = sns.boxplot(x=link['source'], y=link['time'])
# plt.show()
# figB = sns.boxplot(x=link['target'], y=link['time'])
# plt.show()
# hue=link['edge'],

In [ ]:
figX = sns.histplot(x=link['time'], stat="count", bins=100)
figX.set(xlabel=timeLabel)
plt.show()

In [ ]:
figC = sns.boxplot(x=link['edge'], y=link['time'])
plt.xticks(rotation=50, ha="right", fontsize=7)
figC.set(ylabel=timeLabel)
hoverText = mplcursors.cursor(figC, hover=True)
# hoverText.connect("add", lambda sel: sel.annotation.set_text(
#     'Edge {},{}'.format(link['edge'][sel.target[0]], sel.target[1])))
plt.show()

In [ ]:
figBox = sns.boxplot(y=link['time'], showfliers=False)
plt.xticks(rotation=50, ha="right", fontsize=7)
figBox.set(ylabel=timeLabel)
hoverText = mplcursors.cursor(figBox, hover=True)
plt.show()

In [ ]:
figY = sns.histplot(x=link['edge'], stat="count")
figY.set(xlabel='Edge')
hoverText = mplcursors.cursor(figY, hover=True)
plt.xticks(rotation=50, ha="right", fontsize=7)
plt.show()

In [ ]:
# g = sns.FacetGrid(link, col="source",  row="target")
# g = g.map(sns.histplot, 'time')

In [ ]:
# to find the top frequent edges
n = 10
link['edge'].value_counts()[:n]